In [ ]:
import xgboost as xgb

In [ ]:
import pandas as pd

In [ ]:
X = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/X.csv')
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')
Y = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Y.csv')

In [ ]:
X.drop(['TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END'],axis=1,inplace=True)

In [ ]:
Xtest.drop(['TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END'],axis=1,inplace=True)

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
import numpy as np
import time

In [ ]:
for a in X.columns:
    scaler = StandardScaler()
    X[a] = scaler.fit_transform(X[a])
    Xtest[a] = scaler.fit_transform(Xtest[a])

In [ ]:
for a in Xtest.columns:
    scaler = StandardScaler()
    Xtest[a] = scaler.fit_transform(Xtest[a])

In [ ]:
def abs_MAPE_error(y_pred,y_true):
    return np.mean(np.mean(np.abs(y_pred-y_true)/np.abs(y_true)) )
def abs_MAPE_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)/np.abs(y_true)
def abs_error(y_pred,y_true):
    return np.mean(np.mean(np.abs(y_pred-y_true)))  
def abs_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)

In [ ]:
error_list = []
Ytrain_all = pd.DataFrame()
XGBR = xgb.XGBRegressor(max_depth = 3,learning_rate=0.1,n_estimators=500)
TRAIN_TST_C = ['SB00', 'SB01', 'SB02', 'SB03', 'SB04', 'SB05']
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values,Y.values[:,ind] )
    y_true = Y.values[:,ind]
    y_pred = XGBR.predict(X.values)   
    time2 = time.time()  
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))
    
    Ytrain =  XGBR.predict(X.values)
    Ytrain_df = pd.DataFrame(Ytrain)
    Ytrain_all = pd.concat((Ytrain_all,Ytrain_df),axis = 1)   
print(np.mean(error_list))

In [ ]:
#%%
Ytrain_true = pd.DataFrame(Y)
Y_error = abs_MAPE_error_element(Ytrain_all.values,Ytrain_true.values  )  
Good_ind =  Y_error.sum(axis=1).argsort()[0:np.int(0.90*len(X))]
#%%

In [ ]:
XGBR = xgb.XGBRegressor(max_depth = 5,learning_rate=0.01,n_estimators=1600,reg_alpha=1,reg_lambda=0)
Ytest_all = pd.DataFrame()
error_list = []
Ytrain_all = pd.DataFrame()
XGBR = xgb.XGBRegressor(max_depth = 3,learning_rate=0.1,n_estimators=500)
TRAIN_TST_C = ['SB00', 'SB01', 'SB02', 'SB03', 'SB04', 'SB05']
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values[Good_ind],Y.values[Good_ind,ind] )
    y_true = Y.values[Good_ind,ind]
    y_pred = XGBR.predict(X.values[Good_ind])
    time2 = time.time()    
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))
    Ytest = XGBR.predict(Xtest.values)
    Ytest_df = pd.DataFrame(Ytest)
    Ytest_all = pd.concat((Ytest_all,Ytest_df),axis = 1)    
print(np.mean(error_list))

In [ ]:
Ytest_all

In [ ]:
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')

In [ ]:
Ytest_all.columns = [0,1,2,3,4,5]

In [ ]:
result = pd.concat([Ytest_all,Xtest],axis=1)

In [ ]:
result = result.loc[:,[0,1,2,3,4,5,'TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END']]

In [ ]:
import datetime

In [ ]:
result['TRN_END'] = result['TRN_STA'].apply(lambda x :
str((datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours=2))))

In [ ]:
result

In [151]:
sub_result = pd.DataFrame()
for x in range(result.shape[0]):
    t = result.loc[x,:]
    for y in range(6):
        end_time = str(datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*(y+1)))
        start_time = str((datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*y)))
        timewindow = '['+start_time+','+end_time+')'
        each_result = pd.DataFrame({'intersection_id':[t['INTERSECTION_ID']],'tollgate_id':[t['TOLLGATE_ID']]
        ,'time_window':[timewindow] ,'avg_travel_time':[t[y]]})  
        sub_result = pd.concat((sub_result,each_result),axis=0)

In [152]:
sub_result

,avg_travel_time,intersection_id,time_window,tollgate_id
0,52.479847,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",2
0,170.029053,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",2
0,48.159882,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",2
0,41.245922,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",2
0,51.864281,A,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",2
0,56.417545,A,"[2016-10-18 09:40:00,2016-10-18 10:00:00)",2
0,152.707840,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",3
0,143.090836,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",3
0,147.442184,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",3
0,146.661728,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",3


In [153]:
cols = ['intersection_id','tollgate_id','time_window','avg_travel_time']
sub_result = sub_result.ix[:, cols]

In [155]:
sub_result.to_csv('XGBOOST1.csv',index=False,)

504